In [1]:
import os
import cv2
import numpy as np
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tifffile
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Conv2D, MaxPooling2D, Lambda
import matplotlib.pylab as plt
from keras.optimizers import Adam
from tensorflow.keras import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sns
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.metrics import Recall, Precision, BinaryAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import VGG19
from tensorflow.keras.optimizers import Adam

In [3]:
from google.colab import drive
drive.mount('/content/drive')

!ls "/content/drive/My Drive"

Mounted at /content/drive
 1		      'test_1506 (2)'	 val_1506	  'work1506_4 (1)'
 13_07_2023_6_16.tif   test_new		'val_1506 (1)'	   worknew_1
 13_07_2023_6_41.png   tiff		'val_1506 (2)'	   worknew_2
 archive.zip	      'tiff (1)'	 val_new	   worknew_3
 coco.names	       tiff_3		 work1506_1	   worknew_4
'Colab Notebooks'      tiff_4		'work1506_1 (1)'   yolov4.cfg
 datasets	       tiff_5		 work1506_2	   yolov4.weights
 image.jpg	       tiff_6		'work1506_2 (1)'  'Лабораторная работа 4'
 PH2Dataset.rar        train_1506	 work1506_3	  'Лабораторная работа 4.zip'
 test_1506	      'train_1506 (1)'	'work1506_3 (1)'   Сбер
'test_1506 (1)'       'train_1506 (2)'	 work1506_4	   Снимки_tif


Объявляем пути к файлам с исходными данными

In [4]:
base_dir = '/content/drive/MyDrive/tiff/tiff'

Объявляем гипер-параметры

In [5]:
# размеры изображения
img_width, img_height = 122,122
# размерность тензора
input_shape = (img_width, img_height,3)
# количество эпох
epochs = 50
# размер мини-выборки
batch_size = 64

Функция визуализации результатов обучения моделей

In [6]:
def plot_and_save_results(model_vgg, history, i):

    test_loss, test_accuracy, test_recall, test_precision = model_vgg.evaluate(test_generator_aug)
    print('\nВывод метрик:\n')
    print('\nОбщая точность предсказания по всем классам accuracy на тестовом наборе данных для модели ', i, ':', test_accuracy)
    print('\nПотери на тестовом наборе данных для модели ', i, ':', test_loss)
    print('\nПолнота recall на тестовом наборе данных для модели ', i, ':', test_recall)
    print('\nТочность precision на тестовом наборе данных для модели ', i, ':', test_precision)

# График точности на обучающей и валидационной выборке
    plt.figure(figsize=(10,6), dpi=90)
    plt.plot(history.history['binary_accuracy'])
    plt.plot(history.history['val_binary_accuracy'])
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig(f"/content/drive/MyDrive/Снимки_tif/weights/accuracy_png_model_{i}.png", dpi=300)
    plt.show()

# График потерь на обучающей и валидационной выборке
    plt.figure(figsize=(10,6), dpi=90)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig(f"/content/drive/MyDrive/Снимки_tif/weights/loss_png_model_{i}.png", dpi=300)
    plt.show()

    # Получение предсказаний модели на тестовом наборе данных
    predictions = model_vgg.predict(test_generator_aug)

    # Преобразование вероятностей в метки классов
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = test_generator_aug.classes

    # Создаем кросс-матрицу
    cm = confusion_matrix(true_labels, predicted_labels)

    # Визуализируем кросс-матрицу с помощью тепловой карты
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion Matrix Model {i}')
    plt.show()
    plt.savefig(f"/content/drive/MyDrive/Снимки_tif/weights/matrix_png_model_{i}.png", dpi=300)


Функция динамического изменения параметра learning rate

In [7]:
def lr_schedule(epoch, lr):
  if epoch % 15 == 0 and epoch != 0:
    lr = lr * 0.1
  return lr

In [8]:
lr_scheduler = LearningRateScheduler(lr_schedule)

Функция создания модели нейронной сети

In [ ]:
# Загрузка предобученной модели VGG19 без fully connected слоя
base_model_vgg = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

  # Замораживаем все слои
for layer in base_model_vgg.layers:
  layer.trainable = False

  # Размораживаем верхние слои (например, с 15-го слоя и выше)
for layer in base_model_vgg.layers[15:]:
  layer.trainable = True

# Создание новой модели на основе предобученной модели
model_vgg = Sequential()
model_vgg.add(base_model_vgg)
model_vgg.add(Flatten())
model_vgg.add(Dense(1024, activation='relu'))
model_vgg.add(BatchNormalization())
model_vgg.add(Dropout(0.4))
model_vgg.add(Dense(512, activation='relu'))
model_vgg.add(BatchNormalization())
model_vgg.add(Dropout(0.4))
model_vgg.add(Dense(256, activation='relu'))
model_vgg.add(BatchNormalization())
model_vgg.add(Dropout(0.4))

model_vgg.add(Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l2(0.01)))

model_vgg.compile(optimizer=Adam(learning_rate=0.00001),
                  loss='binary_crossentropy',
                  metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

model_vgg.summary()

In [ ]:
train_folders = ['work_1', 'work_2', 'work_3', 'work_4']

i = 1
for folder in train_folders:

    train_dir = os.path.join(base_dir, folder, 'train')
    validation_dir = os.path.join(base_dir, folder, 'val')
    test_dir = os.path.join(base_dir, folder, 'test')

    train_datagen_aug = ImageDataGenerator()
    validation_datagen_aug = ImageDataGenerator()
    test_datagen_aug = ImageDataGenerator()

    train_generator_aug = train_datagen_aug.flow_from_directory(
        train_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary'
    )
    test_generator_aug = test_datagen_aug.flow_from_directory(
        test_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary'
    )

    validation_generator_aug = validation_datagen_aug.flow_from_directory(
        validation_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary'
    )


    history_vgg_aug = model_vgg.fit(
        train_generator_aug,
        steps_per_epoch=train_generator_aug.samples // train_generator_aug.batch_size,
        validation_data=validation_generator_aug,
        validation_steps=validation_generator_aug.samples // validation_generator_aug.batch_size,
        epochs=epochs,
        callbacks=[lr_scheduler]
    )

    model_vgg.save_weights('/content/drive/MyDrive/Снимки_tif/weights/model_vgg19_{}.weights.h5'.format(i))


    plot_and_save_results(model_vgg, history_vgg_aug, i)

    i += 1